Email Automation

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import tkinter as tk
from tkinter import messagebox
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load dataset
file_path = 'credit_card_fraud_dataset.csv'
data = pd.read_csv(file_path)

for i in ['MerchantCategory', 'TransactionLocation', 'CardType', 'DeviceType', 'TransactionChannel']:
    data[i] = data[i].str.strip()

# Data Preprocessing
# Convert categorical variables to numeric
label_encoders = {}
for column in ['MerchantCategory', 'TransactionLocation', 'CardType', 'DeviceType', 'TransactionChannel']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Feature Scaling
scaler = StandardScaler()
data[['TransactionAmount', 'PreviousTransactionAmount']] = scaler.fit_transform(
    data[['TransactionAmount', 'PreviousTransactionAmount']])

# Handling class imbalance using SMOTE
X = data[['TransactionAmount', 'PreviousTransactionAmount', 'MerchantCategory', 'TransactionLocation', 'CardType', 'DeviceType', 'TransactionChannel']]
y = data['FraudLabel']
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Model Training
# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Email Configuration (update with your own SMTP server details)
EMAIL_HOST = 'smtp.gmail.com'
EMAIL_PORT = 587  # TLS port
EMAIL_ADDRESS = 'admi.credicardfraudproject@gmail.com'  # Your Gmail address
EMAIL_PASSWORD = 'credit@123'  # Your Gmail password or app-specific password

# GUI using Tkinter
class FraudDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Credit Card Fraud Detection")

        self.entries = {}
        columns = ['TransactionAmount', 'PreviousTransactionAmount', 
                   'MerchantCategory', 'TransactionLocation', 'CardType', 'DeviceType', 'TransactionChannel']

        for i, column in enumerate(columns):
            label = tk.Label(root, text=column)
            label.grid(row=i, column=0)
            entry = tk.Entry(root)
            entry.grid(row=i, column=1)
            self.entries[column] = entry

        # Adding email entry field
        label_email = tk.Label(root, text="Email Address")
        label_email.grid(row=len(columns), column=0)
        self.entry_email = tk.Entry(root)
        self.entry_email.grid(row=len(columns), column=1)

        self.knn_button = tk.Button(root, text="Predict with KNN", command=self.predict_knn)
        self.knn_button.grid(row=len(columns) + 1, column=0)

        self.lr_button = tk.Button(root, text="Predict with LR", command=self.predict_lr)
        self.lr_button.grid(row=len(columns) + 1, column=1)

    def get_input_data(self):
        input_data = {}
        for column, entry in self.entries.items():
            value = entry.get()
            if column in ['MerchantCategory', 'TransactionLocation', 'CardType', 'DeviceType', 'TransactionChannel']:
                value = label_encoders[column].transform([value])[0]  
            input_data[column] = float(value)
            if column in ['TransactionAmount', 'PreviousTransactionAmount']:
                input_data[column] = float(value)
        
        input_data['Email'] = self.entry_email.get()  # Get the email address from the entry field
        return input_data

    def predict_knn(self):
        input_data = self.get_input_data()
        input_df = pd.DataFrame([input_data])
        input_df = input_df.drop(columns= ['Email'])
        input_df[['TransactionAmount', 'PreviousTransactionAmount']] = scaler.transform(
            input_df[['TransactionAmount', 'PreviousTransactionAmount']])
        prediction = knn.predict(input_df)[0]
        self.show_result(prediction)

    def predict_lr(self):
        input_data = self.get_input_data()
        input_df = pd.DataFrame([input_data])
        input_df = input_df.drop(columns= ['Email'])
        input_df[['TransactionAmount', 'PreviousTransactionAmount']] = scaler.transform(
            input_df[['TransactionAmount', 'PreviousTransactionAmount' ]])
        prediction = lr.predict(input_df)[0]
        self.show_result(prediction)

    def show_result(self, prediction):
        result = "Fraudulent" if prediction == 1 else "Non-Fraudulent"
        messagebox.showinfo("Prediction Result", f"The transaction is predicted to be: {result}")
        
        if prediction == 1:  # If predicted as fraudulent, send an email
            self.send_email(self.get_input_data()['Email'])  # Pass the email address to send_email

    def send_email(self,recipient_email):
        # Email content
        subject = "Fraudulent Transaction Detected"
        body = "A fraudulent credit card transaction has been detected."

        # Create the email
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = recipient_email
        msg['Subject'] = subject

        msg.attach(MIMEText(body, 'plain'))

        # Send the email
        try:
            # Connect to the server
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            
            # Login to your email account
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            
            # Send the email
            text = msg.as_string()
            server.sendmail(EMAIL_ADDRESS, recipient_email, text)
            
            # Disconnect from the server
            server.quit()
            messagebox.showinfo("Email Sent", "An email alert has been sent about the fraudulent transaction.")
        except Exception as e:
            messagebox.showerror("Email Error", f"Error sending email: {str(e)}")


if __name__ == "__main__":
    root = tk.Tk()
    app = FraudDetectionApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python312\Lib\tkinter\__init__.py", line 1962, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_9368\1880824591.py", line 104, in predict_knn
    prediction = knn.predict(input_df)[0]
                 ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\sklearn\neighbors\_classification.py", line 271, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\sklearn\neighbors\_base.py", line 825, in kneighbors
    X = self._validate_data(X, accept_sparse="csr", reset=False, order="C")
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\sklearn\base.py", line 608, in _validate_data
    self._check_feature_names(X, reset=reset)
  File "c:\Python312\Lib\site-packages\s